# Widening the Space of Similar Values

> A major finding was that the current approaches are considering values that are too similar. This notebook investigates ways to search a wider space.  

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

In [ ]:
# | hide
from pathlib import Path
from typing import Callable, Dict, List, Optional, Iterable, Protocol, Sequence, Tuple, TypeVar, Type

In [ ]:
#| hide
from fastcore.test import *
import math
from matplotlib.axes import Axes
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
import tempfile
import torch
from torch.nn import functional as F
from tqdm.auto import tqdm

In [ ]:
# | hide

from transformer_experiments.common.substring_generator import all_unique_substrings
from transformer_experiments.common.text_analysis import (
    build_next_token_map,
    SubstringFrequencyAnalysis,
    top_nonzero_tokens
)
from transformer_experiments.common.utils import (
    aggregate_by_string_key,
    DataWrapper,
    topk_across_batches,
)
from transformer_experiments.dataset_split import split_text_dataset
from transformer_experiments.datasets.tinyshakespeare import (
    TinyShakespeareDataSet,
)
from transformer_experiments.models.transformer import (
    n_embed,
    n_layer,
    TransformerLanguageModel
)
from transformer_experiments.models.transformer_helpers import (
    unsqueeze_emb,
    EncodingHelpers,
    LogitsWrapper,
    TransformerAccessors
)
from transformer_experiments.trained_models.tinyshakespeare_transformer import (
    create_model_and_tokenizer
)
from transformer_experiments.training_utils import CheckPointer
from transformer_experiments.experiments.block_internals import (
    BlockInternalsAccessors,
    BlockInternalsExperiment,
    BatchedBlockInternalsExperiment,
    BlockInternalsAnalysis,
    batch_cosine_sim,
)
from transformer_experiments.experiments.final_ffwd import FinalFFWDExperiment
from transformer_experiments.experiments.similar_strings import (
    SimilarStringsData,
    SimilarStringsExperiment,
    SimilarStringsResult
)
from transformer_experiments.experiments.logit_lens import LogitLens

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
ts = TinyShakespeareDataSet(cache_file='../artifacts/input.txt')
m, tokenizer = create_model_and_tokenizer(
    saved_model_filename='../artifacts/shakespeare.pt',
    dataset=ts,
    device=device,
)
_, val_data = split_text_dataset(ts.text, tokenizer, train_pct=0.9, device=device)
encoding_helpers = EncodingHelpers(tokenizer, device)
accessors = TransformerAccessors(m, device)

In [ ]:
print(f"device is {device}")

device is cpu


In [ ]:
if list(Path('../artifacts/block_internals_results/large_files/slen10/').glob('*')) == []:
    print("Run `make block_internals_slen10_dataset` in the project root to generate the required dataset")

In [ ]:
strings10 = all_unique_substrings(ts.text, 10)

In [ ]:
exp10 = BatchedBlockInternalsExperiment(
    eh=encoding_helpers,
    accessors=accessors,
    strings=strings10,
    output_dir=Path('../artifacts/block_internals_results/large_files/slen10/'),
    batch_size=10000,
)

In [ ]:
torch.manual_seed(1337)
n_samples = 20000
indices = torch.randperm(len(strings10))[:n_samples]
strings20k = [strings10[i.item()] for i in indices]

In [ ]:
# Create a sample of 500 strings
sample_size = 500
strings_sample = strings20k[:sample_size]

In [ ]:
# TODO: put this in a common component
def get_model_outputs(prompts: Sequence[str], encoding_helpers: EncodingHelpers):
    # Compute the model's predictions:
    tokens = encoding_helpers.tokenize_strings(prompts)
    logits, _ = m(tokens)

    logits = LogitsWrapper(logits, encoding_helpers.tokenizer)
    return [topk_tokens[-1] for topk_tokens in logits.topk_tokens(k=10)]


In [ ]:
model_outputs_sample = get_model_outputs(strings_sample, encoding_helpers)

In [ ]:
prompts_exp = BlockInternalsExperiment(encoding_helpers, accessors, strings_sample)

Start by examining what we get when we ask for a much larger top k values. 

In [ ]:
emb_sims, emb_distances = exp10.strings_with_topk_closest_embeddings(
    prompts_exp.embeddings[:5, :, :],
    k=200,
    largest=True,
    distance_function=batch_cosine_sim,
)

In [ ]:
emb_distances[:10, :], emb_distances[-10:, :]

(tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
         [0.9062, 0.9470, 0.9045, 0.9493, 0.9544],
         [0.9062, 0.9462, 0.8602, 0.9083, 0.9443],
         [0.9053, 0.9429, 0.8593, 0.9082, 0.9064],
         [0.9037, 0.9429, 0.8586, 0.9055, 0.9057],
         [0.9027, 0.9404, 0.8566, 0.9052, 0.9045],
         [0.9020, 0.9027, 0.8562, 0.9044, 0.8996],
         [0.9017, 0.9009, 0.8548, 0.9035, 0.8982],
         [0.8962, 0.9009, 0.8545, 0.9034, 0.8656],
         [0.8651, 0.9008, 0.8532, 0.9032, 0.8631]]),
 tensor([[0.7591, 0.7566, 0.7604, 0.8064, 0.8047],
         [0.7591, 0.7566, 0.7604, 0.8063, 0.8047],
         [0.7591, 0.7566, 0.7603, 0.8063, 0.8042],
         [0.7590, 0.7564, 0.7602, 0.8063, 0.8041],
         [0.7589, 0.7563, 0.7601, 0.8062, 0.8041],
         [0.7589, 0.7563, 0.7601, 0.8061, 0.8040],
         [0.7588, 0.7557, 0.7601, 0.8061, 0.8039],
         [0.7587, 0.7557, 0.7600, 0.8060, 0.8039],
         [0.7587, 0.7557, 0.7599, 0.8060, 0.8038],
         [0.7587, 0.7557, 0.7

In [ ]:
for j in range(10):
    print(f"{'   '.join([repr(emb_sims[i][j]) for i in range(len(emb_sims))])}")

print()
for j in range(-10, 0):
    print(f"{'   '.join([repr(emb_sims[i][j]) for i in range(len(emb_sims))])}")


'is dreams,'   'by present'   's eyes may'   'eart of ho'   ' man, as I'
'is dream o'   'My present'   's eye, mak'   'eart of mo'   ' men, as I'
'ur dreams,'   'be present'   's eyes in '   'earn of hi'   ' man, as y'
'of dreams,'   'dy present'   'l eyes can'   'ears of ha'   ' man, if I'
'us dreams.'   'my present'   's eyes to '   'park of ho'   'oman, as t'
'he dreams,'   'ry present'   'l eyes gaz'   'eart of ge'   ' men, as i'
'ly dreams,'   'y, present'   'r foes may'   'eard of hi'   ' many as y'
'en dreams,'   'on present'   'r ever may'   'east of yo'   'cian, as I'
'nd dreams,'   't, present'   's eyes do '   'part of hi'   ' son, as t'
'as dream\nS'   'in present'   's eye; tal'   'earn of yo'   ' long as I'

'is presenc'   ' a prisone'   'g over mas'   'efit of se'   ' man: we s'
'is prowess'   'ot prone t'   'l even tak'   'wist of ro'   ' wind as s'
'is be all,'   'is project'   't so I may'   'e is of so'   ' man; all '
'is the mad'   'my person.'   'n thou may'   'gen

In [ ]:
block_idx = 0
proj_sims, proj_distances = exp10.strings_with_topk_closest_proj_outputs(
    block_idx=block_idx,
    t_i=-1,
    queries=prompts_exp.proj_output(block_idx=block_idx)[:5, -1, :],
    k=200,
    largest=True,
    distance_function=batch_cosine_sim,
)

In [ ]:
proj_distances[:10, :], proj_distances[-10:, :]

(tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
         [0.9948, 0.9967, 0.9947, 0.9974, 0.9956],
         [0.9942, 0.9967, 0.9945, 0.9960, 0.9933],
         [0.9937, 0.9964, 0.9921, 0.9958, 0.9897],
         [0.9935, 0.9963, 0.9917, 0.9957, 0.9896],
         [0.9928, 0.9962, 0.9910, 0.9956, 0.9871],
         [0.9922, 0.9950, 0.9905, 0.9955, 0.9857],
         [0.9911, 0.9944, 0.9900, 0.9954, 0.9856],
         [0.9897, 0.9942, 0.9895, 0.9950, 0.9850],
         [0.9891, 0.9939, 0.9894, 0.9943, 0.9846]]),
 tensor([[0.9709, 0.9826, 0.9796, 0.9855, 0.9726],
         [0.9709, 0.9825, 0.9795, 0.9855, 0.9726],
         [0.9709, 0.9824, 0.9794, 0.9854, 0.9726],
         [0.9708, 0.9824, 0.9793, 0.9854, 0.9725],
         [0.9708, 0.9824, 0.9793, 0.9854, 0.9725],
         [0.9708, 0.9823, 0.9793, 0.9854, 0.9724],
         [0.9708, 0.9823, 0.9791, 0.9854, 0.9724],
         [0.9708, 0.9823, 0.9791, 0.9853, 0.9724],
         [0.9707, 0.9823, 0.9791, 0.9853, 0.9723],
         [0.9707, 0.9823, 0.9

In [ ]:
for j in range(10):
    print(f"{'   '.join([repr(proj_sims[i][j]) for i in range(len(proj_sims))])}")

print()
for j in range(-10, 0):
    print(f"{'   '.join([repr(proj_sims[i][j]) for i in range(len(proj_sims))])}")


'is dreams,'   'by present'   's eyes may'   'eart of ho'   ' man, as I'
'ly dreams,'   'my present'   'e case may'   'ster of ho'   ' men, as I'
'en dreams,'   'dy present'   ' sense may'   'ffer to ha'   ' and, as I'
'he dreams,'   'ry present'   'ied as may'   'ruth of ho'   ' not, as I'
'ur dreams,'   'be present'   'ay she may'   'otes of ho'   'o me, as I'
'nd dreams,'   'My present'   'r foes may'   'anes of ho'   'nd I, as I'
'ery beams,'   'y; present'   'So she may'   'oint of ho'   'cian, as I'
'of dreams,'   'in present'   ' haste may'   'yers of ho'   'I am, as t'
"n's beams,"   'is present'   'odesty may'   'ains of ho'   '-day, as I'
'hese arms,'   'im present'   'esence may'   'ound of ho'   '\nAnd, as I'

'sires most'   'ast ungent'   "'s some am"   'lf with ho'   ' be, was l'
'teous mass'   'What scene'   'e they mad'   'tell of hi'   'Look, as I'
'm to kiss,'   'lest scent'   ' am to say'   'Thus to ha'   'wick, as o'
'much amiss'   'ondon sent'   ' early mad'   's o

In [ ]:
block_idx = 0
ffwd_sims, ffwd_distances = exp10.strings_with_topk_closest_ffwd_outputs(
    block_idx=block_idx,
    t_i=-1,
    queries=prompts_exp.ffwd_output(block_idx=block_idx)[:5, -1, :],
    k=200,
    largest=True,
    distance_function=batch_cosine_sim,
)

In [ ]:
ffwd_distances[:10, :], ffwd_distances[-10:, :]

(tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
         [0.9998, 0.9999, 0.9997, 0.9998, 0.9999],
         [0.9998, 0.9998, 0.9996, 0.9998, 0.9997],
         [0.9998, 0.9998, 0.9996, 0.9998, 0.9997],
         [0.9998, 0.9998, 0.9995, 0.9998, 0.9997],
         [0.9997, 0.9998, 0.9995, 0.9998, 0.9996],
         [0.9997, 0.9997, 0.9995, 0.9998, 0.9995],
         [0.9997, 0.9997, 0.9995, 0.9998, 0.9995],
         [0.9996, 0.9997, 0.9995, 0.9997, 0.9994],
         [0.9996, 0.9997, 0.9995, 0.9997, 0.9994]]),
 tensor([[0.9988, 0.9990, 0.9989, 0.9992, 0.9983],
         [0.9988, 0.9990, 0.9989, 0.9992, 0.9983],
         [0.9988, 0.9990, 0.9989, 0.9992, 0.9983],
         [0.9988, 0.9990, 0.9989, 0.9992, 0.9983],
         [0.9988, 0.9990, 0.9989, 0.9992, 0.9983],
         [0.9987, 0.9990, 0.9989, 0.9992, 0.9983],
         [0.9987, 0.9990, 0.9989, 0.9992, 0.9983],
         [0.9987, 0.9990, 0.9989, 0.9992, 0.9983],
         [0.9987, 0.9990, 0.9989, 0.9992, 0.9983],
         [0.9987, 0.9990, 0.9

In [ ]:
for j in range(10):
    print(f"{'   '.join([repr(ffwd_sims[i][j]) for i in range(len(ffwd_sims))])}")

print()
for j in range(-10, 0):
    print(f"{'   '.join([repr(ffwd_sims[i][j]) for i in range(len(ffwd_sims))])}")

'is dreams,'   'by present'   's eyes may'   'eart of ho'   ' man, as I'
'en dreams,'   'my present'   ' sense may'   'ster of ho'   ' men, as I'
'ly dreams,'   'dy present'   'e case may'   'otes of ho'   ' and, as I'
'he dreams,'   'ry present'   'r foes may'   'anes of ho'   ' not, as I'
'nd dreams,'   'My present'   'ied as may'   'ains of ho'   'o me, as I'
'ur dreams,'   'be present'   'ay she may'   'oint of ho'   'nd I, as I'
'of dreams,'   'y, present'   ' bones may'   'ound of ho'   '-day, as I'
'ery beams,'   'y; present'   'So she may'   'fear to ho'   ' but, as I'
'rate arms,'   'im present'   ' grace may'   'ally of ho'   'rd me as I'
'hese arms,'   'in present'   'e more may'   'yers of ho'   '\nYes, as I'

'ck groans,'   'll\nPresent'   'en you say'   'ace our ho'   'r sakes, I'
'te builds,'   's innocent'   'Hope I may'   'he that ho'   'early as I'
'she finds,'   'me Florent'   'e must say'   'Half an ho'   'nes, and I'
'reat loss,'   'their gent'   'n thou may'   'as

## For the 500 Sample Strings, Can we Calculate Cosine Similarity to Every Other String?


In [ ]:
class CosineSimilaritiesExperiment:
    def __init__(
        self,
        exp: BatchedBlockInternalsExperiment,
        output_folder: Path,
    ):
        self.exp = exp
        self.output_folder = output_folder

        self.n_batches = exp.n_batches

    def embedding_sims_filename(self, batch_idx: int):
        return self.output_folder / f'embedding_sims_{batch_idx:03d}.pt'

    def proj_out_sims_filename(self, batch_idx: int, block_idx: int):
        return self.output_folder / f'proj_out_sims_{batch_idx:03d}_{block_idx:02d}.pt'

    def ffwd_out_sims_filename(self, batch_idx: int, block_idx: int):
        return self.output_folder / f'ffwds_out_sims_{batch_idx:03d}_{block_idx:02d}.pt'

    def generate_embedding_sims(self, chunk_size: int, queries: torch.Tensor, disable_progress_bar: bool = False):
        n_chunks = exp10.batch_size // chunk_size

        assert queries.dim() == 2
        n_queries = queries.shape[0]

        def sims_for_chunk(emb_batch: torch.Tensor, chunk_idx: int):
            chunk = emb_batch[chunk_idx*chunk_size:(chunk_idx+1)*chunk_size, :, :]
            actual_chunk_size = chunk.shape[0]
            return F.cosine_similarity(
                chunk.reshape(actual_chunk_size, 1, -1).expand(-1, n_queries, -1),
                queries,
                dim=-1
            )

        for batch_idx in tqdm(range(self.exp.n_batches), disable=disable_progress_bar):
            emb_batch = torch.load(str(self.exp._embeddings_filename(batch_idx=batch_idx)), mmap=True)
            sims = torch.cat([
                sims_for_chunk(emb_batch, i)
                for i in range(n_chunks)
            ], dim=0)
            torch.save(sims, str(self.embedding_sims_filename(batch_idx=batch_idx)))

    def generate_proj_out_sims(self, get_queries: Callable[[int], torch.Tensor], disable_progress_bar=False):
        for batch_idx in tqdm(range(self.exp.n_batches), disable=disable_progress_bar):
            for block_idx in range(n_layer):
                queries = get_queries(block_idx)

                assert queries.dim() == 2
                n_queries = queries.shape[0]

                proj_out_batch = torch.load(str(self.exp._proj_output_filename(batch_idx=batch_idx, block_idx=block_idx)), mmap=True)
                batch_size = proj_out_batch.shape[0]
                sims = F.cosine_similarity(
                    proj_out_batch[:, -1, :].reshape(batch_size, 1, -1).expand(-1, n_queries, -1),
                    queries,
                    dim=-1
                )
                torch.save(sims, str(self.proj_out_sims_filename(batch_idx=batch_idx, block_idx=block_idx)))

    def generate_ffwd_out_sims(self, get_queries: Callable[[int], torch.Tensor], disable_progress_bar=False):
        for batch_idx in tqdm(range(self.exp.n_batches), disable=disable_progress_bar):
            for block_idx in range(n_layer):
                queries = get_queries(block_idx)
                assert queries.dim() == 2
                n_queries = queries.shape[0]

                ffwd_out_batch = torch.load(str(self.exp._ffwd_output_filename(batch_idx=batch_idx, block_idx=block_idx)), mmap=True)
                batch_size = ffwd_out_batch.shape[0]
                sims = F.cosine_similarity(
                    ffwd_out_batch[:, -1, :].reshape(batch_size, 1, -1).expand(-1, n_queries, -1),
                    queries,
                    dim=-1
                )
                torch.save(sims, str(self.ffwd_out_sims_filename(batch_idx=batch_idx, block_idx=block_idx)))



In [ ]:
# Start by doing some analysis on the first 500 strings.
output_folder = exp10.output_dir / 'cosine_sims_000'
output_folder.mkdir(exist_ok=True)

In [ ]:
cos_exp = CosineSimilaritiesExperiment(exp10, output_folder)

In [ ]:
cos_exp.generate_embedding_sims(chunk_size=2000, queries=prompts_exp.embeddings.reshape(sample_size, -1))

In [ ]:
cos_exp.generate_proj_out_sims(get_queries=lambda block_idx: prompts_exp.proj_output(block_idx=block_idx)[:, -1, :])

In [ ]:
cos_exp.generate_ffwd_out_sims(get_queries=lambda block_idx: prompts_exp.ffwd_output(block_idx=block_idx)[:, -1, :])

In [ ]:
# Now generate cosine sims for ffwd output for the remaining 15000 strings
n_all_strings = 20000
sample_size = 500
n_chunks = math.ceil(n_all_strings / sample_size)

for sample_idx in tqdm(range(0, n_chunks)):
    start_idx = sample_idx * sample_size
    end_idx = min(start_idx + sample_size, n_all_strings)

    # Create a sample of 500 strings
    strings = strings20k[start_idx:end_idx]

    output_folder = exp10.output_dir / f'cosine_sims_{sample_idx:03d}'
    output_folder.mkdir(exist_ok=True)

    exp = CosineSimilaritiesExperiment(exp10, output_folder)

    sample_prompts_exp = BlockInternalsExperiment(encoding_helpers, accessors, strings)

    exp.generate_ffwd_out_sims(
        get_queries=lambda block_idx: sample_prompts_exp.ffwd_output(block_idx=block_idx)[:, -1, :],
        disable_progress_bar=True,
    )

  0%|          | 0/40 [00:00<?, ?it/s]

Generation crashed somewhere during sample_idx 26. We'll have to try again to generate the rest but today we'll work with what we have. 

## Analyze the Results

First, collect some stats about the results

In [ ]:
def stats_across_batches(
    get_batch: Callable[[int], torch.Tensor],
    n_batches: int,
    n_queries: int,
):
    mins = torch.zeros(n_queries)
    maxs = torch.zeros(n_queries)
    means = torch.zeros(n_queries)
    s = torch.zeros(n_queries)

    total_count = 0
    for i in range(n_batches):
        batch = get_batch(i)
        batch_size, n_queries_batch = batch.shape
        assert n_queries_batch == n_queries

        mins = torch.minimum(mins, batch.min(dim=0).values)
        maxs = torch.maximum(maxs, batch.max(dim=0).values)

        # Implement Chan, Golub, and LeVeque method
        total_count += batch_size
        delta = batch.mean(dim=0) - means
        means += delta * batch_size / total_count
        s += batch.var(dim=0) * (batch_size - 1) + delta**2 * batch_size * (total_count - batch_size) / total_count

    vars = s / (total_count - 1)
    stds = torch.sqrt(vars)
    return mins, maxs, means, stds

In [ ]:
ffwd_mins, ffwd_maxs, ffwd_means, ffwd_stds = zip(*[
    stats_across_batches(
        get_batch=lambda i: torch.load(str(cos_exp.ffwd_out_sims_filename(batch_idx=i, block_idx=block_idx))),
        n_batches=cos_exp.n_batches,
        n_queries=sample_size,
    )
    for block_idx in range(n_layer)
])

In [ ]:
for block_idx in range(n_layer):
    print(f"Block {block_idx}: ffwd_out mean: {ffwd_means[block_idx].mean():.3f} ± {ffwd_means[block_idx].std():.3f}")

Block 0: ffwd_out mean: 0.801 ± 0.092
Block 1: ffwd_out mean: 0.365 ± 0.042
Block 2: ffwd_out mean: 0.160 ± 0.033
Block 3: ffwd_out mean: 0.072 ± 0.036
Block 4: ffwd_out mean: 0.088 ± 0.059
Block 5: ffwd_out mean: 0.133 ± 0.091


In [ ]:
def filter_across_batches(
    get_batch: Callable[[int], torch.Tensor],
    n_batches: int,
    filter_fn: Callable[[torch.Tensor], torch.Tensor],
    n_queries: int,
):
    total_count = 0
    matching_indices = [[] for _ in range(n_queries)]
    for i in range(n_batches):
        batch = get_batch(i)
        batch_size, n_queries_batch = batch.shape
        assert n_queries_batch == n_queries

        filtered = filter_fn(batch)
        nonzeros = torch.nonzero(filtered)
        for i in range(nonzeros.shape[0]):
            idx_in_batch, query_idx = nonzeros[i, :]
            matching_indices[query_idx.item()].append(total_count + idx_in_batch.item())

        total_count += batch_size

    return matching_indices

In [ ]:
# Tests for filter_across_batches()

batches = [
    torch.tensor([
        [0.0, 0.6, 0.4, 0.3],
        [0.1, 0.3, 0.5, 0.1],
        [0.0, 0.1, 0.8, 0.0],
    ]),
    torch.tensor([
        [0.7, 0.2, 0.6, 0.3],
        [0.1, 0.8, 0.2, 0.8],
    ]),
]

result = filter_across_batches(
    get_batch=lambda i: batches[i],
    n_batches=len(batches),
    filter_fn=lambda batch: batch > 0.5,
    n_queries=4,
)
test_eq(result, [
    [3,],
    [0, 4],
    [2, 3],
    [4],
])

In [ ]:
def filter_result_stats(
    filter_results: List[List[int]],
):
    lens = [len(result) for result in filter_results]
    return {
        'min': min(lens),
        'max': max(lens),
        'mean': np.mean(lens),
        'std': np.std(lens),
    }

In [ ]:
def get_matching_strings(
    filter_result: List[List[int]],
    strings: Sequence[str],
):
    return [
        [
            strings[j]
            for j in filter_result[i]
        ]
        for i in range(len(filter_result))
    ]

In [ ]:
next_token_map10 = build_next_token_map(ts.text, 10, tokenizer.vocab_size, tokenizer.stoi)

In [ ]:
# TODO: put this in a common component
def analyze_simulate_results(sim_freqs, model_outputs):
    assert len(sim_freqs) == len(model_outputs)
    topn_matches = [0 for _ in range(10)]
    topn_matches_any_order = [0 for _ in range(10)]
    for i, sim_freq in enumerate(sim_freqs):
        sim_output = top_nonzero_tokens(sim_freq, encoding_helpers.tokenizer.itos)[:10]
        model_output = model_outputs[i]

        sim_tokens, _ = zip(*sim_output)
        model_tokens, _ = zip(*model_output)

        n = min(len(sim_tokens), len(model_tokens))
        for j in range(n):
            if sim_tokens[j] == model_tokens[j]:
                topn_matches[j] += 1
            if set(sim_tokens[:j+1]) == set(model_tokens[:j+1]):
                topn_matches_any_order[j] += 1

    return topn_matches, topn_matches_any_order


In [ ]:
def analyze_dataset(
    get_batch: Callable[[int], torch.Tensor],
    n_batches: int,
    filter_fn: Callable[[torch.Tensor], torch.Tensor],
    n_queries: int,
    all_strings: Sequence[str],
    next_token_map: Dict[str, torch.Tensor],
    model_outputs: Sequence[Sequence[Tuple[str, float]]],
):
    filter_results = filter_across_batches(
        get_batch=get_batch,
        n_batches=n_batches,
        filter_fn=filter_fn,
        n_queries=n_queries,
    )

    print(filter_result_stats(filter_results))

    filter_results_strings = get_matching_strings(filter_results, all_strings)
    filter_result_freqs = [
        torch.stack([
            next_token_map[matching_string]
            for matching_string in matching_strings
        ]).sum(dim=0)
        for matching_strings in filter_results_strings
    ]

    filter_result_probs = [
        freqs / freqs.sum()
        for freqs in filter_result_freqs
    ]

    topn_matches, topn_matches_any_order = analyze_simulate_results(filter_result_probs, model_outputs)
    for i in range(10):
        print(f"Top {i+1} matches: {topn_matches[i] / sample_size:.3f}")
        print(f"Top {i+1} matches (any order): {topn_matches_any_order[i] / sample_size:.3f}")

    return filter_result_freqs

In [ ]:
block_idx = 5
ffwd5_freqs = analyze_dataset(
    get_batch=lambda batch_idx: torch.load(str(cos_exp.ffwd_out_sims_filename(batch_idx=batch_idx, block_idx=block_idx)), mmap=True),
    n_batches=cos_exp.n_batches,
    filter_fn=lambda batch: batch > 0.95,
    n_queries=sample_size,
    all_strings=exp10.strings,
    next_token_map=next_token_map10,
    model_outputs=model_outputs_sample,
)

{'min': 1, 'max': 16224, 'mean': 922.742, 'std': 2399.3301297312128}
Top 1 matches: 0.786
Top 1 matches (any order): 0.786
Top 2 matches: 0.404
Top 2 matches (any order): 0.440
Top 3 matches: 0.236
Top 3 matches (any order): 0.306
Top 4 matches: 0.186
Top 4 matches (any order): 0.222
Top 5 matches: 0.118
Top 5 matches (any order): 0.164
Top 6 matches: 0.104
Top 6 matches (any order): 0.124
Top 7 matches: 0.066
Top 7 matches (any order): 0.082
Top 8 matches: 0.052
Top 8 matches (any order): 0.054
Top 9 matches: 0.040
Top 9 matches (any order): 0.048
Top 10 matches: 0.034
Top 10 matches (any order): 0.034


In [ ]:
block_idx = 4
ffwd4_freqs = analyze_dataset(
    get_batch=lambda batch_idx: torch.load(str(cos_exp.ffwd_out_sims_filename(batch_idx=batch_idx, block_idx=block_idx)), mmap=True),
    n_batches=cos_exp.n_batches,
    filter_fn=lambda batch: batch > 0.83,
    n_queries=sample_size,
    all_strings=exp10.strings,
    next_token_map=next_token_map10,
    model_outputs=model_outputs_sample,
)

{'min': 1, 'max': 48461, 'mean': 1877.104, 'std': 5356.329997786171}
Top 1 matches: 0.776
Top 1 matches (any order): 0.776
Top 2 matches: 0.456
Top 2 matches (any order): 0.520
Top 3 matches: 0.296
Top 3 matches (any order): 0.382
Top 4 matches: 0.198
Top 4 matches (any order): 0.260
Top 5 matches: 0.174
Top 5 matches (any order): 0.196
Top 6 matches: 0.130
Top 6 matches (any order): 0.176
Top 7 matches: 0.096
Top 7 matches (any order): 0.124
Top 8 matches: 0.078
Top 8 matches (any order): 0.092
Top 9 matches: 0.078
Top 9 matches (any order): 0.080
Top 10 matches: 0.062
Top 10 matches (any order): 0.052


In [ ]:
block_idx = 3
ffwd3_freqs = analyze_dataset(
    get_batch=lambda batch_idx: torch.load(str(cos_exp.ffwd_out_sims_filename(batch_idx=batch_idx, block_idx=block_idx)), mmap=True),
    n_batches=cos_exp.n_batches,
    filter_fn=lambda batch: batch > 0.97,
    n_queries=sample_size,
    all_strings=exp10.strings,
    next_token_map=next_token_map10,
    model_outputs=model_outputs_sample,
)

{'min': 1, 'max': 1260, 'mean': 38.416, 'std': 114.08503383003399}
Top 1 matches: 0.724
Top 1 matches (any order): 0.724
Top 2 matches: 0.256
Top 2 matches (any order): 0.282
Top 3 matches: 0.084
Top 3 matches (any order): 0.098
Top 4 matches: 0.060
Top 4 matches (any order): 0.068
Top 5 matches: 0.046
Top 5 matches (any order): 0.062
Top 6 matches: 0.030
Top 6 matches (any order): 0.022
Top 7 matches: 0.022
Top 7 matches (any order): 0.016
Top 8 matches: 0.018
Top 8 matches (any order): 0.008
Top 9 matches: 0.018
Top 9 matches (any order): 0.016
Top 10 matches: 0.008
Top 10 matches (any order): 0.008


In [ ]:
block_idx = 2
ffwd2_freqs = analyze_dataset(
    get_batch=lambda batch_idx: torch.load(str(cos_exp.ffwd_out_sims_filename(batch_idx=batch_idx, block_idx=block_idx)), mmap=True),
    n_batches=cos_exp.n_batches,
    filter_fn=lambda batch: batch > 0.90,
    n_queries=sample_size,
    all_strings=exp10.strings,
    next_token_map=next_token_map10,
    model_outputs=model_outputs_sample,
)

{'min': 1, 'max': 6046, 'mean': 468.448, 'std': 876.6662804602445}
Top 1 matches: 0.770
Top 1 matches (any order): 0.770
Top 2 matches: 0.454
Top 2 matches (any order): 0.500
Top 3 matches: 0.278
Top 3 matches (any order): 0.360
Top 4 matches: 0.214
Top 4 matches (any order): 0.246
Top 5 matches: 0.136
Top 5 matches (any order): 0.174
Top 6 matches: 0.102
Top 6 matches (any order): 0.130
Top 7 matches: 0.110
Top 7 matches (any order): 0.098
Top 8 matches: 0.062
Top 8 matches (any order): 0.082
Top 9 matches: 0.060
Top 9 matches (any order): 0.072
Top 10 matches: 0.052
Top 10 matches (any order): 0.050


In [ ]:
block_idx = 1
ffwd1_freqs = analyze_dataset(
    get_batch=lambda batch_idx: torch.load(str(cos_exp.ffwd_out_sims_filename(batch_idx=batch_idx, block_idx=block_idx)), mmap=True),
    n_batches=cos_exp.n_batches,
    filter_fn=lambda batch: batch > 0.99,
    n_queries=sample_size,
    all_strings=exp10.strings,
    next_token_map=next_token_map10,
    model_outputs=model_outputs_sample,
)

{'min': 1, 'max': 4263, 'mean': 169.482, 'std': 383.18133784932684}
Top 1 matches: 0.736
Top 1 matches (any order): 0.736
Top 2 matches: 0.394
Top 2 matches (any order): 0.444
Top 3 matches: 0.196
Top 3 matches (any order): 0.270
Top 4 matches: 0.144
Top 4 matches (any order): 0.192
Top 5 matches: 0.122
Top 5 matches (any order): 0.138
Top 6 matches: 0.080
Top 6 matches (any order): 0.072
Top 7 matches: 0.066
Top 7 matches (any order): 0.064
Top 8 matches: 0.046
Top 8 matches (any order): 0.046
Top 9 matches: 0.062
Top 9 matches (any order): 0.048
Top 10 matches: 0.032
Top 10 matches (any order): 0.022


In [ ]:
block_idx = 0
ffwd0_freqs = analyze_dataset(
    get_batch=lambda batch_idx: torch.load(str(cos_exp.ffwd_out_sims_filename(batch_idx=batch_idx, block_idx=block_idx)), mmap=True),
    n_batches=cos_exp.n_batches,
    filter_fn=lambda batch: batch > 0.97,
    n_queries=sample_size,
    all_strings=exp10.strings,
    next_token_map=next_token_map10,
    model_outputs=model_outputs_sample,
)

{'min': 216, 'max': 111556, 'mean': 23359.618, 'std': 29254.668618804688}
Top 1 matches: 0.444
Top 1 matches (any order): 0.444
Top 2 matches: 0.216
Top 2 matches (any order): 0.208
Top 3 matches: 0.142
Top 3 matches (any order): 0.116
Top 4 matches: 0.164
Top 4 matches (any order): 0.130
Top 5 matches: 0.124
Top 5 matches (any order): 0.100
Top 6 matches: 0.124
Top 6 matches (any order): 0.104
Top 7 matches: 0.094
Top 7 matches (any order): 0.060
Top 8 matches: 0.076
Top 8 matches (any order): 0.032
Top 9 matches: 0.076
Top 9 matches (any order): 0.022
Top 10 matches: 0.048
Top 10 matches (any order): 0.016


In [ ]:
total_freqs = [
    (
        0.00110*ffwd0_freqs[i] +
        0.63*ffwd1_freqs[i] +
        0.08*ffwd2_freqs[i] +
        2*ffwd3_freqs[i] +
        3*ffwd4_freqs[i] +
        10*ffwd5_freq
    )
    for i, ffwd5_freq in enumerate(ffwd5_freqs)
]
total_probs = [
    freqs / freqs.sum()
    for freqs in total_freqs
]
topn_matches, topn_matches_any_order = analyze_simulate_results(total_probs, model_outputs_sample)
for i in range(10):
    print(f"Top {i+1} matches: {topn_matches[i] / sample_size:.3f}")
    print(f"Top {i+1} matches (any order): {topn_matches_any_order[i] / sample_size:.3f}")

Top 1 matches: 0.800
Top 1 matches (any order): 0.800
Top 2 matches: 0.502
Top 2 matches (any order): 0.548
Top 3 matches: 0.340
Top 3 matches (any order): 0.420
Top 4 matches: 0.274
Top 4 matches (any order): 0.316
Top 5 matches: 0.208
Top 5 matches (any order): 0.232
Top 6 matches: 0.198
Top 6 matches (any order): 0.218
Top 7 matches: 0.142
Top 7 matches (any order): 0.138
Top 8 matches: 0.096
Top 8 matches (any order): 0.114
Top 9 matches: 0.114
Top 9 matches (any order): 0.094
Top 10 matches: 0.098
Top 10 matches (any order): 0.072


## Incorporate data from the rest of the strings (everything above just looked at first 500)

In [ ]:
n_all_strings = 20000
sample_size = 500
n_chunks = math.ceil(n_all_strings / sample_size)

last_sample_idx = 25  # This is all we have generated. Get rid of this and just iterate to n_chunks-1 when we have all the data.

next_token_map = next_token_map10
all_strings = strings10

ffwd_thresholds = [0.97, 0.99, 0.90, 0.97, 0.83, 0.95]
ffwd_freqs = [[] for _ in range(n_layer)]
model_outputs = []

for sample_idx in tqdm(range(0, last_sample_idx + 1)):
    output_folder = exp10.output_dir / f"cosine_sims_{sample_idx:03d}"
    output_folder.mkdir(exist_ok=True)

    exp = CosineSimilaritiesExperiment(exp10, output_folder)

    start_idx = sample_idx * sample_size
    end_idx = min(start_idx + sample_size, n_all_strings)

    strings = strings20k[start_idx:end_idx]

    model_outputs.extend(get_model_outputs(strings, encoding_helpers))

    for block_idx in range(n_layer):
        get_batch = lambda batch_idx: torch.load(
            str(exp.ffwd_out_sims_filename(batch_idx=batch_idx, block_idx=block_idx)),
            mmap=True,
        )

        filter_results = filter_across_batches(
            get_batch=get_batch,
            n_batches=exp.n_batches,
            filter_fn=lambda batch: batch > ffwd_thresholds[block_idx],
            n_queries=sample_size,
        )
        filter_results_strings = get_matching_strings(filter_results, all_strings)
        filter_result_freqs = [
            torch.stack(
                [
                    next_token_map[matching_string]
                    for matching_string in matching_strings
                ]
            ).sum(dim=0)
            for matching_strings in filter_results_strings
        ]
        ffwd_freqs[block_idx].extend(filter_result_freqs)

ffwd_freqs = torch.stack(
    [torch.stack(ffwd_freqs[block_idx]) for block_idx in range(n_layer)]
)

  0%|          | 0/26 [00:00<?, ?it/s]

In [ ]:
torch.save(ffwd_freqs, str(exp10.output_dir / 'learn_coefficients/ffwd_freqs.pt'))

In [ ]:
ffwd_freqs.shape, len(model_outputs)

(torch.Size([6, 13000, 65]), 13000)

In [ ]:
# Check that we still get the same results for the first 500
hand_rolled_coeffs = torch.tensor([0.00110, 0.63, 0.08, 2, 3, 10]) .unsqueeze(dim=1).unsqueeze(dim=2) # (n_layer, 1, 1)
total_freqs = (ffwd_freqs[:, :500, :] * hand_rolled_coeffs).sum(dim=0)
total_probs = total_freqs / total_freqs.sum(dim=-1, keepdim=True)
topn_matches, topn_matches_any_order = analyze_simulate_results(total_probs, model_outputs_sample[:500])
for i in range(10):
    print(f"Top {i+1} matches: {topn_matches[i] / sample_size:.3f}")
    print(f"Top {i+1} matches (any order): {topn_matches_any_order[i] / sample_size:.3f}")

Top 1 matches: 0.800
Top 1 matches (any order): 0.800
Top 2 matches: 0.502
Top 2 matches (any order): 0.548
Top 3 matches: 0.340
Top 3 matches (any order): 0.420
Top 4 matches: 0.274
Top 4 matches (any order): 0.316
Top 5 matches: 0.208
Top 5 matches (any order): 0.232
Top 6 matches: 0.198
Top 6 matches (any order): 0.218
Top 7 matches: 0.142
Top 7 matches (any order): 0.138
Top 8 matches: 0.096
Top 8 matches (any order): 0.114
Top 9 matches: 0.114
Top 9 matches (any order): 0.094
Top 10 matches: 0.098
Top 10 matches (any order): 0.072


In [ ]:
# Look at it for all samples
hand_rolled_coeffs = torch.tensor([0.00110, 0.63, 0.08, 2, 3, 10]) .unsqueeze(dim=1).unsqueeze(dim=2) # (n_layer, 1, 1)
total_freqs = (ffwd_freqs * hand_rolled_coeffs).sum(dim=0)
total_probs = total_freqs / total_freqs.sum(dim=-1, keepdim=True)
topn_matches, topn_matches_any_order = analyze_simulate_results(total_probs, model_outputs)
for i in range(10):
    print(f"Top {i+1} matches: {topn_matches[i] / ffwd_freqs.shape[1]:.3f}")
    print(f"Top {i+1} matches (any order): {topn_matches_any_order[i] / ffwd_freqs.shape[1]:.3f}")

Top 1 matches: 0.816
Top 1 matches (any order): 0.816
Top 2 matches: 0.506
Top 2 matches (any order): 0.577
Top 3 matches: 0.356
Top 3 matches (any order): 0.413
Top 4 matches: 0.277
Top 4 matches (any order): 0.330
Top 5 matches: 0.222
Top 5 matches (any order): 0.254
Top 6 matches: 0.200
Top 6 matches (any order): 0.220
Top 7 matches: 0.141
Top 7 matches (any order): 0.153
Top 8 matches: 0.125
Top 8 matches (any order): 0.126
Top 9 matches: 0.107
Top 9 matches (any order): 0.104
Top 10 matches: 0.080
Top 10 matches (any order): 0.064


## Learn Coefficients

In [ ]:
class ModelSim(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.coeffs = torch.nn.Parameter(
            torch.randn(n_layer, 1, 1, dtype=torch.float32, requires_grad=True)
        )

    def forward(self, freqs: torch.Tensor, model_output: Optional[torch.Tensor]=None):
        total_freqs = (freqs * self.coeffs).sum(dim=0)
        total_probs = total_freqs / total_freqs.sum(dim=-1, keepdim=True)

        if model_output is not None:
            loss = torch.norm(total_probs - model_output, dim=-1).sum()
        else:
            loss = None


        return total_probs, loss

In [ ]:
eval_iters = 100


@torch.no_grad()
def estimate_loss(
    model: ModelSim, get_batch: Callable[[str], Tuple[torch.Tensor, torch.Tensor]]
):
    out = {}
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)

            _, loss = model(X, Y)

            losses[k] = loss.item()

        out[split] = losses.mean()
    return out

In [ ]:
def train(
    iters: Sequence[int],
    model: ModelSim,
    optimizer: torch.optim.Optimizer,
    get_batch: Callable[[str], Tuple[torch.Tensor, torch.Tensor]],
    checkpointer: CheckPointer,
    eval_interval: int = 500
):
    for step in tqdm(iters):
        optimizer.zero_grad()


        X, Y = get_batch("train")

        _, loss = model(X, Y)

        if step % eval_interval == 0:
            losses = estimate_loss(model, get_batch)
            print(
                f"step {step}, train loss: {losses['train']:.3f}, val_loss {losses['val']:.3f}"
            )
            checkpointer.save_checkpoint(
                step, model, losses["train"], losses["val"]
            )

        # Take a step
        loss.backward()

        optimizer.step()

In [ ]:
def get_batch(batch_size: int, freqs: torch.Tensor, split: str='train', train_pct: float=0.9):
    n = freqs.shape[1]
    assert split in ['train', 'val']
    n_train = int(n * train_pct)
    low = 0 if split == 'train' else n_train
    high = n_train if split == 'train' else n

    batch_indices = torch.randint(low=low, high=high, size=(batch_size,), dtype=torch.long)
    batch_strings = [strings20k[i.item()] for i in batch_indices]

    tokens = encoding_helpers.tokenize_strings(batch_strings)
    logits, _ = m(tokens)
    model_output = F.softmax(logits[:, -1, :], dim=-1)

    return freqs[:, batch_indices, :].clone(), model_output.detach()


In [ ]:
batch_size=500
get_batch_func = lambda split: get_batch(
    batch_size=batch_size, freqs=ffwd_freqs, split=split, train_pct=0.9
)

In [ ]:
_ = torch.manual_seed(1337) # Ensure stable random values
m2 = ModelSim()

In [ ]:
output_dir = exp10.output_dir / 'learn_coefficients'
output_dir.mkdir(exist_ok=True)
checkpointer = CheckPointer(output_dir, 'coeff_model', start_num=0)

In [ ]:
# Start with a pretty high learning rate
learning_rate = 3e-2
optimizer = torch.optim.AdamW(m2.parameters(), lr=learning_rate)

start_iter = 0
iters = range(start_iter, start_iter+1001)
eval_interval=500

In [ ]:
train(iters=iters, model=m2, optimizer=optimizer, get_batch=get_batch_func, checkpointer=checkpointer, eval_interval=eval_interval)

  0%|          | 0/1001 [00:00<?, ?it/s]

step 0, train loss: 241.612, val_loss 237.651
step 500, train loss: 201.557, val_loss 204.529
step 1000, train loss: 135.628, val_loss 137.657


In [ ]:
losses = estimate_loss(m2, get_batch_func)
losses['train'], losses['val']

(tensor(133.8761), tensor(136.0159))

OK, so we are in a pretty good place. I learned from experience that after this, at the same learning rate, the loss gets worse and then bounces around, never getting back to this point. So we'll slow down the learning rate and go a bit more. But first, let's check in on where we're at.

In [ ]:
m2.coeffs.data

tensor([[[ -0.2058]],

        [[-21.7527]],

        [[-10.9828]],

        [[-21.4913]],

        [[ -8.8436]],

        [[ -0.2939]]])

In [ ]:
total_probs, _ = m2(ffwd_freqs)
topn_matches, topn_matches_any_order = analyze_simulate_results(total_probs, model_outputs)
for i in range(10):
    print(f"Top {i+1} matches: {topn_matches[i] / ffwd_freqs.shape[1]:.3f}")
    print(f"Top {i+1} matches (any order): {topn_matches_any_order[i] / ffwd_freqs.shape[1]:.3f}")

Top 1 matches: 0.736
Top 1 matches (any order): 0.736
Top 2 matches: 0.404
Top 2 matches (any order): 0.466
Top 3 matches: 0.284
Top 3 matches (any order): 0.317
Top 4 matches: 0.236
Top 4 matches (any order): 0.263
Top 5 matches: 0.186
Top 5 matches (any order): 0.195
Top 6 matches: 0.166
Top 6 matches (any order): 0.163
Top 7 matches: 0.117
Top 7 matches (any order): 0.112
Top 8 matches: 0.102
Top 8 matches (any order): 0.090
Top 9 matches: 0.095
Top 9 matches (any order): 0.073
Top 10 matches: 0.070
Top 10 matches (any order): 0.041


Not bad but we know we can do better.

In [ ]:
# Where did iters end up?
iters[-1]

1000

In [ ]:
iters = range(1001, 2001)

# Reduce the learning rate by one order of magnitude
learning_rate = 3e-3
optimizer = torch.optim.AdamW(m2.parameters(), lr=learning_rate)

# Train some more
train(iters=iters, model=m2, optimizer=optimizer, get_batch=get_batch_func, checkpointer=checkpointer, eval_interval=eval_interval)

  0%|          | 0/1000 [00:00<?, ?it/s]

step 1500, train loss: 88.177, val_loss 87.328
step 2000, train loss: 86.917, val_loss 86.809


In [ ]:
m2.coeffs.data

tensor([[[-7.7885e-04]],

        [[-2.1236e+01]],

        [[-1.0711e+01]],

        [[-2.1363e+01]],

        [[-8.9933e+00]],

        [[-5.4774e-02]]])

In [ ]:
total_probs, _ = m2(ffwd_freqs)
topn_matches, topn_matches_any_order = analyze_simulate_results(total_probs, model_outputs)
for i in range(10):
    print(f"Top {i+1} matches: {topn_matches[i] / ffwd_freqs.shape[1]:.3f}")
    print(f"Top {i+1} matches (any order): {topn_matches_any_order[i] / ffwd_freqs.shape[1]:.3f}")

Top 1 matches: 0.789
Top 1 matches (any order): 0.789
Top 2 matches: 0.484
Top 2 matches (any order): 0.560
Top 3 matches: 0.347
Top 3 matches (any order): 0.408
Top 4 matches: 0.273
Top 4 matches (any order): 0.331
Top 5 matches: 0.219
Top 5 matches (any order): 0.251
Top 6 matches: 0.196
Top 6 matches (any order): 0.219
Top 7 matches: 0.145
Top 7 matches (any order): 0.152
Top 8 matches: 0.121
Top 8 matches (any order): 0.123
Top 9 matches: 0.107
Top 9 matches (any order): 0.107
Top 10 matches: 0.081
Top 10 matches (any order): 0.066


An improvement! Let's try a lot more iterations at this same learning rate and see where we get.

In [ ]:
# Where did iters end up?
iters[-1]

2000

In [ ]:
# Let's go for 5000 iterations. We can always go back if it overshoots.
iters = range(2001, 7001)

# Train some more
train(iters=iters, model=m2, optimizer=optimizer, get_batch=get_batch_func, checkpointer=checkpointer, eval_interval=eval_interval)

  0%|          | 0/5000 [00:00<?, ?it/s]

step 2500, train loss: 86.974, val_loss 86.748
step 3000, train loss: 86.863, val_loss 86.883
step 3500, train loss: 87.361, val_loss 86.588
step 4000, train loss: 87.380, val_loss 86.652
step 4500, train loss: 85.888, val_loss 85.819
step 5000, train loss: 86.133, val_loss 85.628
step 5500, train loss: 85.582, val_loss 85.125
step 6000, train loss: 84.050, val_loss 84.298
step 6500, train loss: 83.721, val_loss 83.268
step 7000, train loss: 82.888, val_loss 82.894


In [ ]:
m2.coeffs.data

tensor([[[-1.6660e-03]],

        [[-1.1624e+01]],

        [[-5.4355e-01]],

        [[-2.6189e+01]],

        [[-1.3666e+01]],

        [[-6.0726e-02]]])

In [ ]:
total_probs, _ = m2(ffwd_freqs)
topn_matches, topn_matches_any_order = analyze_simulate_results(total_probs, model_outputs)
for i in range(10):
    print(f"Top {i+1} matches: {topn_matches[i] / ffwd_freqs.shape[1]:.3f}")
    print(f"Top {i+1} matches (any order): {topn_matches_any_order[i] / ffwd_freqs.shape[1]:.3f}")

Top 1 matches: 0.800
Top 1 matches (any order): 0.800
Top 2 matches: 0.491
Top 2 matches (any order): 0.565
Top 3 matches: 0.348
Top 3 matches (any order): 0.407
Top 4 matches: 0.276
Top 4 matches (any order): 0.329
Top 5 matches: 0.220
Top 5 matches (any order): 0.255
Top 6 matches: 0.199
Top 6 matches (any order): 0.218
Top 7 matches: 0.141
Top 7 matches (any order): 0.153
Top 8 matches: 0.124
Top 8 matches (any order): 0.121
Top 9 matches: 0.106
Top 9 matches (any order): 0.104
Top 10 matches: 0.078
Top 10 matches (any order): 0.063


In [ ]:
# Where did iters end up?
iters[-1]

7000

In [ ]:
# Let's go for 5000 iterations. We can always go back if it overshoots.
iters = range(7001, 12001)

# Train some more
train(iters=iters, model=m2, optimizer=optimizer, get_batch=get_batch_func, checkpointer=checkpointer, eval_interval=eval_interval)

  0%|          | 0/5000 [00:00<?, ?it/s]

step 7500, train loss: 81.342, val_loss 82.018
step 8000, train loss: 81.164, val_loss 81.277
step 8500, train loss: 80.309, val_loss 80.065
step 9000, train loss: 78.922, val_loss 79.470
step 9500, train loss: 77.919, val_loss 78.434
step 10000, train loss: 77.058, val_loss 78.159
step 10500, train loss: 77.470, val_loss 78.453
step 11000, train loss: 77.198, val_loss 78.682
step 11500, train loss: 77.445, val_loss 78.351
step 12000, train loss: 77.037, val_loss 78.661


In [ ]:
m2.coeffs.data

tensor([[[-1.4626e-03]],

        [[-3.7772e-01]],

        [[ 1.3287e-02]],

        [[-3.2255e+01]],

        [[-1.5003e+01]],

        [[-6.6595e-02]]])

In [ ]:
total_probs, _ = m2(ffwd_freqs)
topn_matches, topn_matches_any_order = analyze_simulate_results(total_probs, model_outputs)
for i in range(10):
    print(f"Top {i+1} matches: {topn_matches[i] / ffwd_freqs.shape[1]:.3f}")
    print(f"Top {i+1} matches (any order): {topn_matches_any_order[i] / ffwd_freqs.shape[1]:.3f}")

Top 1 matches: 0.810
Top 1 matches (any order): 0.810
Top 2 matches: 0.496
Top 2 matches (any order): 0.567
Top 3 matches: 0.347
Top 3 matches (any order): 0.403
Top 4 matches: 0.271
Top 4 matches (any order): 0.324
Top 5 matches: 0.217
Top 5 matches (any order): 0.252
Top 6 matches: 0.196
Top 6 matches (any order): 0.215
Top 7 matches: 0.142
Top 7 matches (any order): 0.152
Top 8 matches: 0.122
Top 8 matches (any order): 0.121
Top 9 matches: 0.104
Top 9 matches (any order): 0.099
Top 10 matches: 0.079
Top 10 matches (any order): 0.061


In [ ]:
# Where did iters end up?
iters[-1]

12000

In [ ]:
# Let's go for 1000 more
iters = range(12001, 13001)

# Train some more
train(iters=iters, model=m2, optimizer=optimizer, get_batch=get_batch_func, checkpointer=checkpointer, eval_interval=eval_interval)

  0%|          | 0/1000 [00:00<?, ?it/s]

step 12500, train loss: 77.357, val_loss 79.174
step 13000, train loss: 77.685, val_loss 78.382


Seems like we're kinda stuck here. Let's see what happens if we go back to a bigger learning rate. 

In [ ]:
# Where did iters end up?
iters[-1]

13000

In [ ]:
iters = range(13001, 14001)

# Reduce the learning rate by one order of magnitude
learning_rate = 3e-2
optimizer = torch.optim.AdamW(m2.parameters(), lr=learning_rate)

# Train some more
train(iters=iters, model=m2, optimizer=optimizer, get_batch=get_batch_func, checkpointer=checkpointer, eval_interval=eval_interval)

  0%|          | 0/1000 [00:00<?, ?it/s]

step 13500, train loss: 135.310, val_loss 135.358
step 14000, train loss: 122.897, val_loss 124.560


OK that was a big step backwards. Let's go back to the checkpoint for step 12000.

In [ ]:
checkpoint = torch.load(checkpointer.output_dir / 'coeff_model_000024.pt')
checkpoint['iters'], checkpoint['train_loss'], checkpoint['val_loss']

(12000, tensor(77.0366), tensor(78.6612))

In [ ]:
m2.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [ ]:
estimate_loss(m2, get_batch_func)

{'train': tensor(76.3558), 'val': tensor(78.6847)}

OK, so we're back to a good state. From here, let's try a smaller learning rate and see if we can get to a better place.

In [ ]:
iters = range(12001, 22001)

# Reduce the learning rate by one order of magnitude
learning_rate = 3e-4
optimizer = torch.optim.AdamW(m2.parameters(), lr=learning_rate)

# Train some more
train(iters=iters, model=m2, optimizer=optimizer, get_batch=get_batch_func, checkpointer=checkpointer, eval_interval=eval_interval)

  0%|          | 0/10000 [00:00<?, ?it/s]

step 12500, train loss: 77.424, val_loss 77.532
step 13000, train loss: 77.258, val_loss 77.458
step 13500, train loss: 77.586, val_loss 78.413
step 14000, train loss: 77.379, val_loss 77.972
step 14500, train loss: 77.640, val_loss 77.830
step 15000, train loss: 77.147, val_loss 77.483
step 15500, train loss: 77.133, val_loss 78.076
step 16000, train loss: 77.453, val_loss 77.666
step 16500, train loss: 77.320, val_loss 77.908
step 17000, train loss: 77.376, val_loss 77.925
step 17500, train loss: 77.313, val_loss 78.315
step 18000, train loss: 77.748, val_loss 78.471
step 18500, train loss: 76.673, val_loss 78.184
step 19000, train loss: 77.181, val_loss 78.109
step 19500, train loss: 77.044, val_loss 78.242
step 20000, train loss: 76.708, val_loss 77.908
step 20500, train loss: 77.322, val_loss 78.222
step 21000, train loss: 77.078, val_loss 77.792
step 21500, train loss: 76.784, val_loss 77.895
step 22000, train loss: 76.980, val_loss 77.926


OK, seems like this is stagnating. Try slowing down the learning rate even further and going for one more round.

In [ ]:
# Where did iters end up?
iters[-1]

22000

In [ ]:
iters = range(22001, 32001)

# Reduce the learning rate by one order of magnitude
learning_rate = 3e-5
optimizer = torch.optim.AdamW(m2.parameters(), lr=learning_rate)

# Train some more
train(iters=iters, model=m2, optimizer=optimizer, get_batch=get_batch_func, checkpointer=checkpointer, eval_interval=eval_interval)

  0%|          | 0/10000 [00:00<?, ?it/s]

step 22500, train loss: 76.870, val_loss 77.868
step 23000, train loss: 76.529, val_loss 77.656
step 23500, train loss: 76.755, val_loss 78.100
step 24000, train loss: 77.780, val_loss 78.070
step 24500, train loss: 77.234, val_loss 77.849
step 25000, train loss: 77.190, val_loss 77.891
step 25500, train loss: 77.453, val_loss 77.203
step 26000, train loss: 76.894, val_loss 77.747
step 26500, train loss: 77.383, val_loss 77.889
step 27000, train loss: 77.054, val_loss 77.632
step 27500, train loss: 77.147, val_loss 77.393
step 28000, train loss: 77.582, val_loss 77.413
step 28500, train loss: 77.349, val_loss 77.526
step 29000, train loss: 77.317, val_loss 78.182
step 29500, train loss: 76.733, val_loss 77.809
step 30000, train loss: 77.529, val_loss 77.429
step 30500, train loss: 77.185, val_loss 77.459
step 31000, train loss: 77.191, val_loss 77.592
step 31500, train loss: 77.590, val_loss 78.024
step 32000, train loss: 77.452, val_loss 78.473


OK, looks like this is still kind of stuck in this region and maybe can't go anywhere better from here. What if we initialize with the hand-rolled weights and see if it can improve that? 

In [ ]:
m3 = ModelSim()
m3.coeffs = torch.nn.Parameter(torch.tensor([0.00110, 0.63, 0.08, 2, 3, 10]).unsqueeze(dim=1).unsqueeze(dim=2))
m3.coeffs.shape

torch.Size([6, 1, 1])

In [ ]:
output_dir = exp10.output_dir / 'learn_coefficients'
output_dir.mkdir(exist_ok=True)
checkpointer = CheckPointer(output_dir, 'coeff_model_hand_rolled', start_num=0)

In [ ]:
# Begin with a moderate learning rate
learning_rate = 3e-4
optimizer = torch.optim.AdamW(m3.parameters(), lr=learning_rate)

start_iter = 0
iters = range(start_iter, start_iter+1001)
eval_interval=500

In [ ]:
train(iters=iters, model=m3, optimizer=optimizer, get_batch=get_batch_func, checkpointer=checkpointer, eval_interval=eval_interval)

  0%|          | 0/1001 [00:00<?, ?it/s]

step 0, train loss: 78.948, val_loss 80.359
step 500, train loss: 76.345, val_loss 77.676
step 1000, train loss: 76.179, val_loss 77.503


In [ ]:
m3.coeffs.data

tensor([[[ 6.0409e-04]],

        [[ 3.6633e-01]],

        [[-1.7641e-02]],

        [[ 2.1913e+00]],

        [[ 3.0050e+00]],

        [[ 1.0185e+01]]])

In [ ]:
total_probs, _ = m3(ffwd_freqs)
topn_matches, topn_matches_any_order = analyze_simulate_results(total_probs, model_outputs)
for i in range(10):
    print(f"Top {i+1} matches: {topn_matches[i] / ffwd_freqs.shape[1]:.3f}")
    print(f"Top {i+1} matches (any order): {topn_matches_any_order[i] / ffwd_freqs.shape[1]:.3f}")

Top 1 matches: 0.817
Top 1 matches (any order): 0.817
Top 2 matches: 0.504
Top 2 matches (any order): 0.574
Top 3 matches: 0.355
Top 3 matches (any order): 0.413
Top 4 matches: 0.275
Top 4 matches (any order): 0.330
Top 5 matches: 0.221
Top 5 matches (any order): 0.253
Top 6 matches: 0.198
Top 6 matches (any order): 0.218
Top 7 matches: 0.141
Top 7 matches (any order): 0.153
Top 8 matches: 0.124
Top 8 matches (any order): 0.124
Top 9 matches: 0.105
Top 9 matches (any order): 0.102
Top 10 matches: 0.078
Top 10 matches (any order): 0.063


Almost as good as what I had hand-rolled. Let's go a little more and see if we can improve it. 

In [ ]:
iters[-1]

1000

In [ ]:
learning_rate = 3e-5
optimizer = torch.optim.AdamW(m3.parameters(), lr=learning_rate)

iters = range(1001, 11001)

In [ ]:
train(iters=iters, model=m3, optimizer=optimizer, get_batch=get_batch_func, checkpointer=checkpointer, eval_interval=eval_interval)

  0%|          | 0/10000 [00:00<?, ?it/s]

step 1500, train loss: 76.471, val_loss 76.904
step 2000, train loss: 75.632, val_loss 77.666
step 2500, train loss: 76.099, val_loss 76.458
step 3000, train loss: 75.324, val_loss 76.594
step 3500, train loss: 76.446, val_loss 76.955
step 4000, train loss: 75.880, val_loss 76.352
step 4500, train loss: 76.183, val_loss 76.920
step 5000, train loss: 75.911, val_loss 76.729
step 5500, train loss: 75.490, val_loss 76.433
step 6000, train loss: 75.511, val_loss 77.376
step 6500, train loss: 75.577, val_loss 75.980
step 7000, train loss: 76.168, val_loss 76.554
step 7500, train loss: 75.267, val_loss 76.620
step 8000, train loss: 76.097, val_loss 76.439
step 8500, train loss: 75.934, val_loss 76.713
step 9000, train loss: 75.824, val_loss 76.684
step 9500, train loss: 75.885, val_loss 76.549
step 10000, train loss: 75.604, val_loss 76.377
step 10500, train loss: 75.899, val_loss 76.442
step 11000, train loss: 75.453, val_loss 76.470


In [ ]:
m3.coeffs.data

tensor([[[ 2.5622e-04]],

        [[ 1.7031e-01]],

        [[-7.9133e-03]],

        [[ 2.3232e+00]],

        [[ 2.9156e+00]],

        [[ 1.0321e+01]]])

In [ ]:
total_probs, _ = m3(ffwd_freqs)
topn_matches, topn_matches_any_order = analyze_simulate_results(total_probs, model_outputs)
for i in range(10):
    print(f"Top {i+1} matches: {topn_matches[i] / ffwd_freqs.shape[1]:.3f}")
    print(f"Top {i+1} matches (any order): {topn_matches_any_order[i] / ffwd_freqs.shape[1]:.3f}")

Top 1 matches: 0.818
Top 1 matches (any order): 0.818
Top 2 matches: 0.507
Top 2 matches (any order): 0.576
Top 3 matches: 0.356
Top 3 matches (any order): 0.413
Top 4 matches: 0.275
Top 4 matches (any order): 0.330
Top 5 matches: 0.221
Top 5 matches (any order): 0.254
Top 6 matches: 0.198
Top 6 matches (any order): 0.220
Top 7 matches: 0.142
Top 7 matches (any order): 0.154
Top 8 matches: 0.123
Top 8 matches (any order): 0.125
Top 9 matches: 0.106
Top 9 matches (any order): 0.102
Top 10 matches: 0.079
Top 10 matches (any order): 0.063


This is actually the best result so far. 

## Cosine Sims for Length 256 Strings


In [ ]:
strings256 = all_unique_substrings(ts.text, 256)

In [ ]:
torch.manual_seed(1337)
sample_size = 500
indices256 = torch.randperm(len(strings256))[:sample_size]
strings256_sample = [strings256[i.item()] for i in indices256]

In [ ]:
model_outputs_sample256 = get_model_outputs(strings256_sample, encoding_helpers)

In [ ]:
prompts_exp256 = BlockInternalsExperiment(encoding_helpers, accessors, strings256_sample)

In [ ]:
class CosineSimilaritiesForFinalFFWDExperiment:
    def __init__(
        self,
        exp: FinalFFWDExperiment,
        output_folder: Path,
    ):
        self.exp = exp
        self.output_folder = output_folder

        self.n_batches = exp.n_batches

    def ffwd_out_sims_filename(self, batch_idx: int, block_idx: int):
        return self.output_folder / f'ffwds_out_sims_{batch_idx:04d}_{block_idx:02d}.pt'

    def generate_ffwd_out_sims(self, get_queries: Callable[[int], torch.Tensor], disable_progress_bar=False):
        block_idx = n_layer - 1
        for batch_idx in tqdm(range(self.exp.n_batches), disable=disable_progress_bar):
            queries = get_queries(block_idx)
            assert queries.dim() == 2
            n_queries = queries.shape[0]

            ffwd_out_batch = torch.load(str(self.exp._ffwd_output_filename(batch_idx=batch_idx, block_idx=block_idx)), mmap=True)
            batch_size = ffwd_out_batch.shape[0]
            sims = F.cosine_similarity(
                ffwd_out_batch.reshape(batch_size, 1, -1).expand(-1, n_queries, -1),
                queries,
                dim=-1
            )
            torch.save(sims, str(self.ffwd_out_sims_filename(batch_idx=batch_idx, block_idx=block_idx)))




In [ ]:
ffwd_exp256 = FinalFFWDExperiment(
    eh=encoding_helpers,
    accessors=accessors,
    strings=strings256,
    output_dir=Path('../artifacts/block_internals_results/large_files/slen256'),
    batch_size=400,
)

In [ ]:
output_folder = ffwd_exp256.output_dir / 'cosine_sims'
output_folder.mkdir(exist_ok=True)

In [ ]:
cos_exp256 = CosineSimilaritiesForFinalFFWDExperiment(ffwd_exp256, output_folder)

In [ ]:
cos_exp256.generate_ffwd_out_sims(get_queries=lambda block_idx: prompts_exp256.ffwd_output(block_idx=block_idx)[:, -1, :])

  0%|          | 0/2788 [00:00<?, ?it/s]

In [ ]:
next_token_map256 = build_next_token_map(ts.text, 256, tokenizer.vocab_size, tokenizer.stoi)

In [ ]:
block_idx = n_layer - 1
ffwd256 = filter_across_batches(
    get_batch=lambda batch_idx: torch.load(str(cos_exp256.ffwd_out_sims_filename(batch_idx=batch_idx, block_idx=block_idx)), mmap=True),
    n_batches=cos_exp256.n_batches,
    filter_fn=lambda batch: batch > 0.91,
    n_queries=sample_size,
)
filter_result_stats(ffwd256)

{'min': 1, 'max': 38780, 'mean': 1561.9, 'std': 4162.8430373964375}

In [ ]:
ffwd256_strings = get_matching_strings(ffwd256, strings256)

In [ ]:
ffwd256_freqs = [
    torch.stack([
        next_token_map256[matching_string]
        for matching_string in matching_strings
    ]).sum(dim=0)
    for matching_strings in ffwd256_strings
]
ffwd256_probs = [
    freqs / freqs.sum()
    for freqs in ffwd256_freqs
]

In [ ]:
topn_matches, topn_matches_any_order = analyze_simulate_results(ffwd256_probs, model_outputs_sample256)
for i in range(10):
    print(f"Top {i+1} matches: {topn_matches[i] / sample_size:.3f}")
    print(f"Top {i+1} matches (any order): {topn_matches_any_order[i] / sample_size:.3f}")

Top 1 matches: 0.802
Top 1 matches (any order): 0.802
Top 2 matches: 0.362
Top 2 matches (any order): 0.412
Top 3 matches: 0.246
Top 3 matches (any order): 0.298
Top 4 matches: 0.200
Top 4 matches (any order): 0.250
Top 5 matches: 0.156
Top 5 matches (any order): 0.154
Top 6 matches: 0.128
Top 6 matches (any order): 0.148
Top 7 matches: 0.070
Top 7 matches (any order): 0.086
Top 8 matches: 0.044
Top 8 matches (any order): 0.068
Top 9 matches: 0.032
Top 9 matches (any order): 0.038
Top 10 matches: 0.030
Top 10 matches (any order): 0.016


In [ ]:
idx = 0
print(f"Query {idx} ({repr(strings256_sample[idx])}): ")
result = ffwd256[idx]
for j in result[:20]:
    print(f"  {repr(strings256[j])}")


Query 0 (" see my shame in him.\nThou art a widow; yet thou art a mother,\nAnd hast the comfort of thy children left thee:\nBut death hath snatch'd my husband from mine arms,\nAnd pluck'd two crutches from my feeble limbs,\nEdward and Clarence. O, what cause have I,\nThin"): 
  " so.\n\nSICINIUS:\nLet them assemble,\nAnd on a safer judgment all revoke\nYour ignorant election; enforce his pride,\nAnd his old hate unto you; besides, forget not\nWith what contempt he wore the humble weed,\nHow in his suit he scorn'd you; but your loves,\nThin"
  "I'ld have beaten him like a dog, but for\ndisturbing the lords within.\n\nAUFIDIUS:\nWhence comest thou? what wouldst thou? thy name?\nWhy speak'st not? speak, man: what's thy name?\n\nCORIOLANUS:\nIf, Tullus,\nNot yet thou knowest me, and, seeing me, dost not\nThin"
  "e not--to save my life, for if\nI had fear'd death, of all the men i' the world\nI would have 'voided thee, but in mere spite,\nTo be full quit of those my banishers,\nStand I bef

In [ ]:
class TensorBatchIterator:
    def __init__(self, n_batches: int, get_batch: Callable[[int], torch.Tensor]):
        self.n_batches = n_batches
        self.get_batch = get_batch

        self.next_batch_idx = 0
        self.current_batch: Optional[torch.Tensor] = None
        self.idx_within_batch = 0

        self._load_next_batch()

    def _load_next_batch(self):
        if self.next_batch_idx >= self.n_batches:
            raise StopIteration()

        self.current_batch = self.get_batch(self.next_batch_idx)
        self.idx_within_batch = 0
        self.next_batch_idx += 1

    def __next__(self):
        if self.current_batch is None:
            raise StopIteration()

        if self.idx_within_batch >= self.current_batch.shape[0]:
            self._load_next_batch()
            if self.current_batch is None:
                raise StopIteration()

        result = self.current_batch[self.idx_within_batch, :]
        self.idx_within_batch += 1
        return result

class EmbeddingCosineSims:
    def __init__(self, exp: CosineSimilaritiesExperiment):
        self.exp = exp

    def __iter__(self):
        return TensorBatchIterator(
            n_batches=self.exp.n_batches,
            get_batch=lambda batch_idx: torch.load(str(self.exp.embedding_sims_filename(batch_idx=batch_idx)), mmap=True)
        )



In [ ]:
emb_sims = EmbeddingCosineSims(cos_exp)
for i, sim in enumerate(emb_sims):
    print(i, sim.shape)

In [ ]:
sims = next(iter(emb_sims))
sims.shape

torch.Size([500])